In [3]:
%pip install -U openmim
!mim install mmengine
!mim install "mmcv>=2.0.0,<2.2.0"


!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

%pip install -e .

Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.5.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.5.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.4/471.4 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mmcv: filename=mmcv-2.1.0-cp310-cp310-linux_x86_64.whl size=38670359 sha256=fd72dc83f6fc8f661abd8c66b198f61ad6a7998495182729fc3d4d7ebaea4564
  Stored in directory: /root/.cache/pip/wheels/8b/09/35/94a7f7ba6a00e3810abd0492340e4cbba0ff3d443120a94120
Successfully built mmcv
Cloning into 'mmdetection'...
remote: Enumerating objects: 38023, done.
remote: Total 38023 (delta 0), reused 0 (delta 0), pack-reused 38023 (from 1)
Receiving objects: 100% (38023/38023), 63.18 MiB | 25.56 MiB/s, done.
Resolving deltas: 100% (26233/26233), done.
/content/mmdetection/mmdetection
Obtaining file:///content/mmdetection/mmdetection
  Preparing metadata (setup.py) ... done
  Attempting uninstall

In [18]:
import torch
import mmdet
import mmcv
import mmengine

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
config_aquarium = """
# Inherit and overwrite part of the config based on this config
_base_ = '/content/mmdetection/mmdet/configs/rtmdet/rtmdet_tiny_8xb32_300e_coco.py'

data_root = '/content/drive/MyDrive/Aquarium Combined.v2-raw-1024.coco-mmdetection' # dataset root

train_batch_size_per_gpu = 4
train_num_workers = 2

max_epochs = 20
stage2_num_epochs = 1
base_lr = 0.00008


metainfo = {
    'classes': ('fish', 'jellyfish', 'penguin', 'puffin', 'shark', 'starfish', 'stingray'),
    'palette': []
}

train_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='/train/images'),
        ann_file='/train/annotation/_annotations.coco.json'))

val_dataloader = dict(
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='/valid/images'),
        ann_file='/valid/annotation/_annotations.coco.json'))

test_dataloader = val_dataloader

val_evaluator = dict(ann_file=data_root + '/valid/annotation/_annotations.coco.json')

test_evaluator = val_evaluator

model = dict(bbox_head=dict(num_classes=7))

# learning rate
param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=1.0e-5,
        by_epoch=False,
        begin=0,
        end=10),
    dict(
        # use cosine lr from 10 to 20 epoch
        type='CosineAnnealingLR',
        eta_min=base_lr * 0.05,
        begin=max_epochs // 2,
        end=max_epochs,
        T_max=max_epochs // 2,
        by_epoch=True,
        convert_to_iter_based=True),
]

train_pipeline_stage2 = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='RandomResize',
        scale=(640, 640),
        ratio_range=(0.1, 2.0),
        keep_ratio=True),
    dict(type='RandomCrop', crop_size=(640, 640)),
    dict(type='YOLOXHSVRandomAug'),
    dict(type='RandomFlip', prob=0.5),
    dict(type='Pad', size=(640, 640), pad_val=dict(img=(114, 114, 114))),
    dict(type='PackDetInputs')
]

# optimizer
optim_wrapper = dict(
    _delete_=True,
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=base_lr, weight_decay=0.05),
    paramwise_cfg=dict(
        norm_decay_mult=0, bias_decay_mult=0, bypass_duplicate=True))

default_hooks = dict(
    checkpoint=dict(
        interval=5,
        max_keep_ckpts=2,  # only keep latest 2 checkpoints
        save_best='auto'
    ),
    logger=dict(type='LoggerHook', interval=5))

custom_hooks = [
    dict(
        type='PipelineSwitchHook',
        switch_epoch=max_epochs - stage2_num_epochs,
        switch_pipeline=train_pipeline_stage2)
]

# load COCO pre-trained weight
#load_from = 'https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'

train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=max_epochs, val_interval=1)
visualizer = dict(vis_backends=[dict(type='LocalVisBackend'),dict(type='TensorboardVisBackend')])
"""

with open('/content/drive/MyDrive/configs/rtmdet/rtmdet_tiny_1xb4-20e_aquarium.py', 'w') as f:
    f.write(config_aquarium)

In [26]:
!python /content/mmdetection/tools/train.py /content/drive/MyDrive/configs/rtmdet/rtmdet_tiny_1xb4-20e_aquarium.py

Traceback (most recent call last):
  File "/content/mmdetection/tools/train.py", line 8, in <module>
    from mmengine.runner import Runner
  File "/usr/local/lib/python3.10/dist-packages/mmengine/runner/__init__.py", line 2, in <module>
    from ._flexible_runner import FlexibleRunner
  File "/usr/local/lib/python3.10/dist-packages/mmengine/runner/_flexible_runner.py", line 14, in <module>
    from mmengine._strategy import BaseStrategy
  File "/usr/local/lib/python3.10/dist-packages/mmengine/_strategy/__init__.py", line 4, in <module>
    from .base import BaseStrategy
  File "/usr/local/lib/python3.10/dist-packages/mmengine/_strategy/base.py", line 19, in <module>
    from mmengine.model.wrappers import is_model_wrapper
  File "/usr/local/lib/python3.10/dist-packages/mmengine/model/__init__.py", line 6, in <module>
    from .base_model import BaseDataPreprocessor, BaseModel, ImgDataPreprocessor
  File "/usr/local/lib/python3.10/dist-packages/mmengine/model/base_model/__init__.py", l

In [25]:
import os

# Check if the training script exists
print(os.path.exists('/content/mmdetection/tools/train.py'))  # Should return True

# Check if the config file exists
print(os.path.exists('/content/drive/MyDrive/configs/rtmdet/rtmdet_tiny_1xb4-20e_aquarium.py'))  # Should return True


True
True


In [22]:
!pip list | grep mmdetection
!pip list | grep mmengine

mmdet                              3.3.0               /content/mmdetection/mmdetection
mmengine                           0.9.0


In [21]:
!pip install mmengine==0.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: mmengine
    Found existing installation: mmengine 0.10.5
    Uninstalling mmengine-0.10.5:
      Successfully uninstalled mmengine-0.10.5
